# ***Simulating Data for Klaus High-Dimensional Data Visualization***
### This script is for simulating multi-dimensional data for Klaus. The raw 3D is grabbed from https://www.digitalrocksportal.org/projects/6 The 3D data is cropped from the original data.

In [1]:
%matplotlib qt
import numpy as np
import numpy.random as nr
import scipy.stats as ss
import tifffile, os, time
import multiprocess as mp
from scipy.ndimage import zoom
import matplotlib.pyplot as plt

***generate fake data based on the orginal 3D structure***

In [2]:
ph1_cen = 41
ph1_std = 25

ph2_cen = 68
ph2_std = 25

ph3_cen = 82
ph3_std = 25

ph4_cen = 141
ph4_std = 25

y_p1 = ss.norm(ph1_cen, ph1_std).pdf
y_p2 = ss.norm(ph2_cen, ph2_std).pdf
y_p3 = ss.norm(ph3_cen, ph3_std).pdf
y_p4 = ss.norm(ph4_cen, ph4_std).pdf

p = np.ndarray([4])

def ratio(gray_val):   
    p[0] = y_p1(gray_val)
    p[1] = y_p2(gray_val)
    p[2] = y_p3(gray_val)
    p[3] = y_p4(gray_val)
    ps = p.sum()
    # print(ps)
    return list(p[:i].sum()/ps for i in range(1, 5))
    
def discriminator(gray_val):
    c1 = c2 = c3 = c4 = 0
    p1, p2, p3, p4 = ratio(gray_val)
    for ii in range(1000):
        seat = nr.random()
        if seat<p1:
            c1 += 1
        elif seat<p2:
            c2 += 1
        elif seat<p3:
            c3 += 1
        else:
            c4 += 1
    return c1, c2, c3, c4
    # return nr.random()

# if __name__ == "__main__":
data = tifffile.imread("/media/xiao_usb/originating/256x256x256.tif").reshape([-1])
n_cpu = os.cpu_count()
dim = data.shape
print(data.shape, data[100])
results = []

print(time.asctime())
with mp.Pool(maxtasksperchild=10) as pool:
    rlt = pool.map(discriminator, [data[ii] for ii in np.int32(np.arange(dim[0]))])
      
pool.close()
pool.join()

print(time.asctime())
print(len(rlt))

results2 = np.array(rlt).reshape([256, 256, 256, 4])

tifffile.imsave("/media/xiao_usb/originating/channel1.tif", results2[:, :, :, 0].astype(np.float32))
tifffile.imsave("/media/xiao_usb/originating/channel2.tif", results2[:, :, :, 1].astype(np.float32))
tifffile.imsave("/media/xiao_usb/originating/channel3.tif", results2[:, :, :, 2].astype(np.float32))
tifffile.imsave("/media/xiao_usb/originating/channel4.tif", results2[:, :, :, 3].astype(np.float32))

(16777216,) 96
Wed Jun 17 17:24:18 2020
Wed Jun 17 17:54:23 2020
16777216


***normalize density maps and scale them to a real NMC sample***

In [3]:
# unit area mass density: ug/cm^2/(xrf_cnt/ic_cnt)
convert_factor = {}
convert_factor['co'] = 7858.034843219578
convert_factor['mn'] = 12533.170403614253
convert_factor['ni'] = 6538.971984538915
convert_factor['zr'] = 1492803.120660955

# xrf-ic conversion: xrf_cnt/ic_cnt
fac_xrf_per_ic = {}
fac_xrf_per_ic['co'] = 9e-4
fac_xrf_per_ic['mn'] = 3e-4
fac_xrf_per_ic['ni'] = 2e-3
fac_xrf_per_ic['zr'] = 1e-6

rel_mass_den = {}

for ii in fac_xrf_per_ic.keys():
    rel_mass_den[ii] = fac_xrf_per_ic[ii] * convert_factor[ii]
    print(ii, rel_mass_den[ii])

co 7.07223135889762
mn 3.7599511210842755
ni 13.07794396907783
zr 1.492803120660955


In [4]:
### in xrf simulation, the slice thickness is assumed to 50nm. The voxel size is assumed 50x50x50nm^3
### the original image values are normalized to 1, which corresponds the typical mass density of each element
### the unit of den maps is g/cm^3
ori = {}
ori['co'] = tifffile.imread("/media/xiao_usb/originating/channel1.tif")
ori['mn'] = tifffile.imread("/media/xiao_usb/originating/channel2.tif")
ori['ni'] = tifffile.imread("/media/xiao_usb/originating/channel3.tif")
ori['zr'] = tifffile.imread("/media/xiao_usb/originating/channel4.tif")

den = {}
for ii in ori.keys():
    ori[ii] /= np.max(ori[ii])
    den[ii] = ori[ii]*rel_mass_den[ii]*(1e7/50)*1e-6
    print(ii, rel_mass_den[ii]*(1e7/50)*1e-6)
    

co 1.4144462717795239
mn 0.7519902242168551
ni 2.615588793815566
zr 0.298560624132191


---
> ## Elements' attenuation parameters
---
> -------- E ---------- delta ---------------- beta -------------- density   
> Co: 8400. ||| 3.59271144E-06 ||| 4.72017234E-07 ||| 1.414    
> Mn: 8400. ||| 1.98677685E-06 ||| 2.13132296E-07 ||| 0.752     
> Ni: 8400. ||| 5.56073564E-06 ||| 9.95139771E-07 ||| 2.616    
> Zr: 8400. ||| 7.64205595E-07 ||| 4.06926191E-08 ||| 0.299    

>> matter attenuation is calculated as  
>>   exp(-2 * 2pi * beta * z / lambda)  
>>   According to Beer's law  
>>   exp(-u * z)  
>> the linear attentuation coefficient is calculated as  
>>   u = 4 * pi * beta/lambda  
---

In [5]:
tifffile.imsave("/media/xiao_usb/originating/co_xrf.tif", den['co'].astype(np.float32))
tifffile.imsave("/media/xiao_usb/originating/mn_xrf.tif", den['mn'].astype(np.float32))
tifffile.imsave("/media/xiao_usb/originating/ni_xrf.tif", den['ni'].astype(np.float32))
tifffile.imsave("/media/xiao_usb/originating/zr_xrf.tif", den['zr'].astype(np.float32))

In [6]:
### unit 1/cm
lin_att_coef = {}
lin_att_coef['co'] = 4*np.pi*4.72017234E-07/(12.4/8.4)*1e8
lin_att_coef['mn'] = 4*np.pi*2.13132296E-07/(12.4/8.4)*1e8
lin_att_coef['ni'] = 4*np.pi*9.95139771E-07/(12.4/8.4)*1e8
lin_att_coef['zr'] = 4*np.pi*4.06926191E-08/(12.4/8.4)*1e8

In [27]:
### assume 20nm pixel size in tomography simulation
print(lin_att_coef)

{'co': 401.81423701607304, 'mn': 181.43318661268165, 'ni': 847.1328989837574, 'zr': 34.640416743453436}


In [28]:
np.exp(-lin_att_coef['ni']*1e-3)

0.42864213214728286

In [7]:
### zoom 3D images of each element to reflect the fact that TXM and XRF CT scans usually have different voxel sizes
zoom_den = {}
zoom_den['co'] = zoom(den['co'], 2.5)
zoom_den['mn'] = zoom(den['mn'], 2.5)
zoom_den['ni'] = zoom(den['ni'], 2.5)
zoom_den['zr'] = zoom(den['zr'], 2.5)
print(zoom_den['co'].shape)

(640, 640, 640)


In [8]:
tifffile.imsave("/media/xiao_usb/originating/co_zoomed_xrf.tif", zoom_den['co'].astype(np.float32))
tifffile.imsave("/media/xiao_usb/originating/mn_zoomed_xrf.tif", zoom_den['mn'].astype(np.float32))
tifffile.imsave("/media/xiao_usb/originating/ni_zoomed_xrf.tif", zoom_den['ni'].astype(np.float32))
tifffile.imsave("/media/xiao_usb/originating/zr_zoomed_xrf.tif", zoom_den['zr'].astype(np.float32))



In [37]:
plt.figure(1)
plt.imshow(zoom_den['co'][320, ...])
plt.figure(2)
plt.imshow(den['co'][128, ...])

In [9]:
### calculate u*t map that simulate a tomography reconstruction
tomo = {}

for ii in zoom_den.keys():
    tomo[ii] = zoom_den[ii]/np.max(zoom_den[ii])*lin_att_coef[ii]*(20/1e7)

In [10]:
tifffile.imsave("/media/xiao_usb/originating/tomo.tif", (tomo['co']+tomo['mn']+tomo['ni']+tomo['zr']).astype(np.float32))